# PCA Analysis for `anaphora-1` models

In [ ]:
import os
import sys

import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from hydra import compose, initialize, initialize_config_dir, initialize_config_module
from hydra.utils import instantiate
from matplotlib import pyplot as plt
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import LightningDataModule, LightningModule
from sklearn.decomposition import PCA

# Needed for under-the-hood loading of models by framework
sys.path.append("../")

In [ ]:
def get_model_from_ckpt(exp_path):
    config_name = "config"
    wandb_path = "wandb/latest-run/files/"
    exp_dir = os.path.abspath(os.path.join("../", exp_path))
    ckpt_dir = os.path.abspath(os.path.join("../", exp_path, "checkpoints"))
    ckpt_path = os.path.join(ckpt_dir, "last.ckpt")
    saved_wandb_dir = os.path.abspath(os.path.join("../", exp_path, wandb_path))
    saved_cfg_dir = os.path.join(exp_dir, ".hydra")

    assert os.path.exists(f"{saved_cfg_dir}/{config_name}.yaml")
    assert os.path.exists(f"{saved_wandb_dir}/{config_name}.yaml")

    cfgs = {}

    with initialize_config_dir(version_base="1.1", config_dir=saved_cfg_dir):
        cfg = compose(config_name=config_name)
        cfgs["hydra"] = cfg

    with initialize_config_dir(version_base="1.1", config_dir=saved_wandb_dir):
        cfg = compose(config_name=config_name)
        cfgs["wandb"] = cfg

    model = create_model(cfgs)
    model = model.__class__.load_from_checkpoint(ckpt_path)
    model.eval()

    datamodule = create_datamodule(cfgs)

    return model, datamodule


def create_datamodule(cfgs):
    datamodule_cfg = cfgs["hydra"].datamodule
    data_dir = cfgs["wandb"]["datamodule/data_dir"].value

    with open_dict(datamodule_cfg):
        datamodule_cfg.data_dir = data_dir

    datamodule: LightningDataModule = instantiate(datamodule_cfg)
    return datamodule


def create_model(cfgs):
    model_cfg = cfgs["hydra"].model
    dec_vocab_size = cfgs["wandb"]["model/dec_vocab_size"].value
    enc_vocab_size = cfgs["wandb"]["model/enc_vocab_size"].value
    dec_EOS_idx = cfgs["wandb"]["model/dec_EOS_idx"].value
    with open_dict(model_cfg):
        model_cfg.dec_vocab_size = dec_vocab_size
        model_cfg.enc_vocab_size = enc_vocab_size
        model_cfg.dec_EOS_idx = dec_EOS_idx

    model: LightningModule = instantiate(model_cfg)

    return model

In [ ]:
# Iterate through the entire dataset.
# Compute the last-state encodings of each input,
# and stack the results into a (D,H)-sized tensor,
# where D = length of dataset and H = encoder hidden
# size.
#
# Perform k=2 PCA on this to create a (D,2)-sized tensor
# for analysis


def get_analysis_df(model, datamodule, dataloaders):

    data_encodings = None
    data_inputs = None
    data_preds = None
    data_target = None

    for dl in dataloaders:
        for batch in dl:
            with torch.no_grad():
                batch_enc = model.get_encodings(batch)["encoder_last_state"]
                _, preds, target = model.step(batch)

            if data_encodings is not None:

                data_encodings = torch.cat((data_encodings, batch_enc), dim=0)
                data_preds = torch.cat((data_preds, preds), dim=0)
                data_target = torch.cat((data_target, target), dim=0)

                # Pad input tensors if lengths are wrong
                i_size = max(data_inputs.shape[1], batch[0].shape[1])
                i_delta = i_size - min(data_inputs.shape[1], batch[0].shape[1])
                if i_size > data_inputs.shape[1]:
                    data_inputs = F.pad(data_inputs, (0, i_delta), "constant", 0)
                    batch_0 = batch[0]
                elif i_size > batch[0].shape[1]:
                    batch_0 = F.pad(batch[0], (0, i_delta), "constant", 0)
                else:
                    batch_0 = batch[0]

                data_inputs = torch.cat((data_inputs, batch_0), dim=0)

            else:
                data_encodings = batch_enc
                data_inputs = batch[0]
                data_preds = preds
                data_target = target

    if len(data_encodings.shape) > 2:
        if data_encodings.shape[1] > 1:
            # only look at the last layer
            data_encodings = data_encodings[:, 1, :]
        data_encodings = torch.squeeze(data_encodings)

    i_labels = [
        datamodule.data_train.dataset.convert_tokens_to_string(k, col="source")
        for _, k in enumerate(data_inputs)
    ]
    i_labels = [" ".join(l) for l in i_labels]

    t_labels = [
        datamodule.data_train.dataset.convert_tokens_to_string(k, col="target")
        for _, k in enumerate(data_target)
    ]
    t_labels = [" ".join(l) for l in t_labels]

    p_labels = [
        datamodule.data_train.dataset.convert_tokens_to_string(k, col="target")
        for _, k in enumerate(data_preds)
    ]
    p_labels = [" ".join(l) for l in p_labels]

    pt_pca = torch.pca_lowrank(data_encodings, q=2)
    pt_reduced_enc = (data_encodings @ pt_pca[2]).detach()

    df = pd.DataFrame(
        {
            "input": i_labels,
            "target": t_labels,
            "prediction": p_labels,
            "pca1": pt_reduced_enc[:, 0],
            "pca2": pt_reduced_enc[:, 1],
        }
    )

    return df

## Anaphora-1

In [ ]:
anaphora_srn_model, anaphora_datamodule = get_model_from_ckpt(
    exp_path="outputs/anaphora-1/2022-09-12_16-14-42"
)
anaphora_datamodule.setup()

In [ ]:
anaphora_df = get_analysis_df(
    anaphora_srn_model,
    anaphora_datamodule,
    [
        anaphora_datamodule.train_dataloader(),
        anaphora_datamodule.val_dataloader(),
        anaphora_datamodule.test_dataloader(),
        anaphora_datamodule.gen_dataloader(),
    ],
)

In [ ]:
anaphora_df["himself"] = anaphora_df["input"].str.contains("himself")
anaphora_df["herself"] = anaphora_df["input"].str.contains("herself")
anaphora_df["refl"] = anaphora_df["himself"] | anaphora_df["herself"]
anaphora_df["alice"] = anaphora_df["input"].str.contains("alice")
anaphora_df["bob"] = anaphora_df["input"].str.contains("bob")
anaphora_df["claire"] = anaphora_df["input"].str.contains("claire")
anaphora_df["knows"] = anaphora_df["input"].str.contains("knows")
anaphora_df["likes"] = anaphora_df["input"].str.contains("likes")
anaphora_df["sees"] = anaphora_df["input"].str.contains("sees")
anaphora_df["alice_refl"] = anaphora_df["alice"] & anaphora_df["refl"]
anaphora_df["claire_refl"] = anaphora_df["claire"] & anaphora_df["refl"]
anaphora_df["intrans"] = anaphora_df["input"].str.contains("<PAD>")

int_to_refl = {0: "non-reflexive", 1: "herself", 2: "himself"}
anaphora_df["refl_type"] = anaphora_df["herself"].apply(int)
anaphora_df["refl_type"] += anaphora_df["himself"].apply(int).apply(lambda x: 2 * x)
anaphora_df["refl_type"] = anaphora_df["refl_type"].apply(lambda x: int_to_refl[x])

### Intransitive vs Transitive

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="intrans", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

### Reflexives

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="refl_type", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

### Alice vs Bob vs Claire

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="alice", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="bob", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="claire", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

### Knows vs Likes vs Sees

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="knows", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="likes", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="sees", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

### Alice-REFL vs Claire-REFL

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="alice_refl", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="claire_refl", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

## GRU Anaphora-1

In [ ]:
anaphora_srn_model, anaphora_datamodule = get_model_from_ckpt(
    exp_path="outputs/anaphora-1/2022-10-03_16-28-04"
)
anaphora_datamodule.setup()

In [ ]:
anaphora_df = get_analysis_df(
    anaphora_srn_model,
    anaphora_datamodule,
    [
        anaphora_datamodule.train_dataloader(),
        anaphora_datamodule.val_dataloader(),
        anaphora_datamodule.test_dataloader(),
        anaphora_datamodule.gen_dataloader(),
    ],
)

In [ ]:
anaphora_df["himself"] = anaphora_df["input"].str.contains("himself")
anaphora_df["herself"] = anaphora_df["input"].str.contains("herself")
anaphora_df["refl"] = anaphora_df["himself"] | anaphora_df["herself"]
anaphora_df["alice"] = anaphora_df["input"].str.contains("alice")
anaphora_df["bob"] = anaphora_df["input"].str.contains("bob")
anaphora_df["claire"] = anaphora_df["input"].str.contains("claire")
anaphora_df["knows"] = anaphora_df["input"].str.contains("knows")
anaphora_df["likes"] = anaphora_df["input"].str.contains("likes")
anaphora_df["sees"] = anaphora_df["input"].str.contains("sees")
anaphora_df["alice_refl"] = anaphora_df["alice"] & anaphora_df["refl"]
anaphora_df["claire_refl"] = anaphora_df["claire"] & anaphora_df["refl"]
anaphora_df["bob_refl"] = anaphora_df["bob"] & anaphora_df["refl"]
anaphora_df["intrans"] = anaphora_df["input"].str.contains("<PAD>")

int_to_refl = {0: "non-reflexive", 1: "herself", 2: "himself"}
anaphora_df["refl_type"] = anaphora_df["herself"].apply(int)
anaphora_df["refl_type"] += anaphora_df["himself"].apply(int).apply(lambda x: 2 * x)
anaphora_df["refl_type"] = anaphora_df["refl_type"].apply(lambda x: int_to_refl[x])

### Intransitive vs Transitive

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="intrans", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

### Reflexives

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="refl_type", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

### Alice vs Bob vs Claire

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="alice", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="bob", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="claire", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="claire_refl", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="alice_refl", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=anaphora_df, hue="bob_refl", fit_reg=False)
lm.fig.suptitle("PCA of 49-dim SRN")

## SCAN Add Jump

In [ ]:
scan_srn_model, scan_datamodule = get_model_from_ckpt(
    exp_path="outputs/SCAN (Add Jump)/2022-10-02_11-43-28"
)
scan_datamodule.setup()

In [ ]:
scan_df = get_analysis_df(
    scan_srn_model,
    scan_datamodule,
    [
        scan_datamodule.train_dataloader(),
        scan_datamodule.val_dataloader(),
        scan_datamodule.test_dataloader(),
    ],
)

In [ ]:
scan_df["jump"] = scan_df["input"].str.contains("jump")
scan_df["walk"] = scan_df["input"].str.contains("walk")
scan_df["turn_right"] = scan_df["input"].str.contains("turn right")
scan_df["twice"] = scan_df["input"].str.contains("twice")
scan_df["jump_twice"] = scan_df["jump"] & scan_df["twice"]
scan_df["walk_twice"] = scan_df["walk"] & scan_df["twice"]
scan_df["turn_right_twice"] = scan_df["turn_right"] & scan_df["twice"]

### Twice

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=scan_df, hue="twice", fit_reg=False)
lm.fig.suptitle("PCA of SRN for SCAN (Add Jump)")

### Jump vs Walk vs Turn Right

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=scan_df, hue="jump", fit_reg=False)
lm.fig.suptitle("PCA of SRN for SCAN (Add Jump)")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=scan_df, hue="walk", fit_reg=False)
lm.fig.suptitle("PCA of SRN for SCAN (Add Jump)")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=scan_df, hue="turn_right", fit_reg=False)
lm.fig.suptitle("PCA of SRN for SCAN (Add Jump)")

### {Jump, Walk, Turn Right} Twice

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=scan_df, hue="jump_twice", fit_reg=False)
lm.fig.suptitle("PCA of SRN for SCAN (Add Jump)")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=scan_df, hue="walk_twice", fit_reg=False)
lm.fig.suptitle("PCA of SRN for SCAN (Add Jump)")

In [ ]:
lm = sns.lmplot("pca1", "pca2", data=scan_df, hue="turn_right_twice", fit_reg=False)
lm.fig.suptitle("PCA of SRN for SCAN (Add Jump)")